# Data Workshop 1

**Instructor:** Tommy Stone, Applied Ocean Science (AOS) PhD student (thstone@ucsd.edu)
**TAs:** Austin Green (a7green@ucsd.edu)

This notebook is meant to be a general guide into data analysis with Python. We will focus on loading a NetCDF file  

Lectures notes adapted from Andrea Rodriguez-Marin Freudmann's Data Workshop 1 notebook

## 1 | Importing Libraries and Loading a NetCDF File



### 1.1 Importing Libraries

### 1.2 Loading the data